<a href="https://www.kaggle.com/code/redpolish/customer360-dataset?scriptVersionId=110368442" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats 
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')

In [2]:
customers = pd.read_csv('../input/brazilian-ecommerce/olist_customers_dataset.csv')
geolocation = pd.read_csv('../input/brazilian-ecommerce/olist_geolocation_dataset.csv')
items = pd.read_csv('../input/brazilian-ecommerce/olist_order_items_dataset.csv')
payments = pd.read_csv('../input/brazilian-ecommerce/olist_order_payments_dataset.csv')
reviews = pd.read_csv('../input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
orders = pd.read_csv('../input/brazilian-ecommerce/olist_orders_dataset.csv')
products = pd.read_csv('../input/brazilian-ecommerce/olist_products_dataset.csv')
sellers = pd.read_csv('../input/brazilian-ecommerce/olist_sellers_dataset.csv')
category = pd.read_csv('../input/brazilian-ecommerce/product_category_name_translation.csv')

In [3]:
data = customers.merge(orders, on = 'customer_id', how ='left')
data = data.merge(payments, on = 'order_id')
data = data.merge(items, on = 'order_id')
data = data.merge(reviews, on = 'order_id')
data = data.merge(products, on = 'product_id')
data = data.merge(sellers, on = 'seller_id')

In [4]:
#customer's location
customerdata = data[['customer_unique_id','customer_city','customer_state']].drop_duplicates(subset='customer_unique_id').reset_index(drop=True)
customerdata

,customer_unique_id,customer_city,customer_state
0,861eff4711a542e4b93843c6dd7febb0,franca,SP
1,9eae34bbd3a474ec5d07949ca7de67c0,santarem,PA
2,3c799d181c34d51f6d44bbbc563024db,nova santa rita,RS
3,23397e992b09769faf5e66f9e171a241,mage,RJ
4,567ab47ca4deb92d46dbf54dce07d0a7,angelina,SC
...,...,...,...
94715,9ba58a98da1d0abf0bd1e422e06c9051,sao manuel,SP
94716,5c22dbb9d8b5385953f2f1e6d9d115a7,jordania,MG
94717,6532316a8c1a7c7a569482db073530bb,guarulhos,SP
94718,7f5f43a9394964ac51bc5d1e0204ead4,uruacu,GO


In [5]:
#customer's activity data
activitydata = data.groupby(['customer_unique_id'])['order_purchase_timestamp'].max()
activitydata = pd.DataFrame(data = activitydata).reset_index()
activitydata = activitydata.rename(columns = {'order_purchase_timestamp' : 'last_purchase'})
activitydata['last_purchase']=activitydata[['last_purchase']].apply(pd.to_datetime)

activitydata['last_active'] = (activitydata['last_purchase'].max() - activitydata['last_purchase']).dt.days

def activity (days):
    if days < 31:
        return ("active")
    else:
        return ("inactive")

activitydata['activity_status'] = activitydata.apply(lambda x: activity(x.last_active),axis=1)
activitydata

,customer_unique_id,last_purchase,last_active,activity_status
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,115,inactive
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,118,inactive
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,541,inactive
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,325,inactive
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,292,inactive
...,...,...,...,...
94715,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08 21:00:36,451,inactive
94716,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10 20:07:56,266,inactive
94717,ffff371b4d645b6ecea244b27531430a,2017-02-07 15:49:16,572,inactive
94718,ffff5962728ec6157033ef9805bacc48,2018-05-02 15:17:41,123,inactive


In [6]:
#map purchase time
data['order_purchase_timestamp']=data['order_purchase_timestamp'].apply(pd.to_datetime)

#day and day of week
data['order_purchase_date'] = data['order_purchase_timestamp'].apply(lambda x: x.day)
data['order_purchase_dayofweek'] = data['order_purchase_timestamp'].apply(lambda x: x.dayofweek)
data['order_purchase_dayofweek_name'] = data['order_purchase_timestamp'].apply(lambda x: x.strftime('%a'))
data['order_purchase_month'] = data['order_purchase_timestamp'].apply(lambda x: x.strftime('%B'))
data['order_purchase_year'] = data['order_purchase_timestamp'].apply(lambda x: x.year)

#hour and time
data['order_purchase_hour'] = data['order_purchase_timestamp'].apply(lambda x: x.hour)
hours_bins = [-0.1, 6, 12, 18, 23]
hours_labels = ['Dawn', 'Morning', 'Afternoon', 'Night']
data['order_purchase_time_day'] = pd.cut(data['order_purchase_hour'], hours_bins, labels=hours_labels)

#time of the month
day_bins = [0, 11, 21, 31]
day_labels = ['Early', 'Middle','End']
data['order_purchase_time_month'] = pd.cut(data['order_purchase_date'], day_bins, labels=day_labels)

In [7]:
#most common time to buy
time_to_buy = data[['customer_unique_id','order_purchase_date','order_purchase_dayofweek_name','order_purchase_month','order_purchase_year','order_purchase_time_day','order_purchase_time_month']]
time_to_buy['common_date'] = time_to_buy.groupby('customer_unique_id')['order_purchase_date'].transform(lambda x: x.mode()[0])
time_to_buy['common_day'] = time_to_buy.groupby('customer_unique_id')['order_purchase_dayofweek_name'].transform(lambda x: x.mode()[0])
time_to_buy['common_month'] = time_to_buy.groupby('customer_unique_id')['order_purchase_month'].transform(lambda x: x.mode()[0])
time_to_buy['common_year'] = time_to_buy.groupby('customer_unique_id')['order_purchase_year'].transform(lambda x: x.mode()[0])
time_to_buy['common_day_time'] = time_to_buy.groupby('customer_unique_id')['order_purchase_time_day'].transform(lambda x: x.mode()[0])
time_to_buy['common_month_time'] = time_to_buy.groupby('customer_unique_id')['order_purchase_time_month'].transform(lambda x: x.mode()[0])
#time_to_buy=time_to_buy.drop(columns=['order_purchase_date','order_purchase_dayofweek_name','order_purchase_month','order_purchase_year','order_purchase_time_day','order_purchase_time_month']).drop_duplicates(subset="customer_unique_id").reset_index(inplace=True)
time_to_buy = time_to_buy.drop(columns=['order_purchase_date','order_purchase_dayofweek_name','order_purchase_month','order_purchase_year','order_purchase_time_day','order_purchase_time_month']).drop_duplicates(subset="customer_unique_id").reset_index(drop=True)
time_to_buy

,customer_unique_id,common_date,common_day,common_month,common_year,common_day_time,common_month_time
0,861eff4711a542e4b93843c6dd7febb0,16,Tue,May,2017,Afternoon,Middle
1,9eae34bbd3a474ec5d07949ca7de67c0,9,Thu,November,2017,Dawn,Early
2,3c799d181c34d51f6d44bbbc563024db,7,Sun,May,2017,Night,Early
3,23397e992b09769faf5e66f9e171a241,3,Sat,February,2018,Night,Early
4,567ab47ca4deb92d46dbf54dce07d0a7,23,Thu,March,2017,Afternoon,End
...,...,...,...,...,...,...,...
94715,9ba58a98da1d0abf0bd1e422e06c9051,16,Mon,July,2018,Night,Middle
94716,5c22dbb9d8b5385953f2f1e6d9d115a7,1,Wed,August,2018,Afternoon,Early
94717,6532316a8c1a7c7a569482db073530bb,20,Mon,August,2018,Morning,Middle
94718,7f5f43a9394964ac51bc5d1e0204ead4,18,Sat,August,2018,Night,Middle


In [8]:
#customer orders
count_order = data.groupby('customer_unique_id')[['order_id']].count().reset_index()
avg_item = data.groupby('customer_unique_id')[['order_item_id']].mean().reset_index()
customerorder = count_order.merge(avg_item,on='customer_unique_id',how='left')
customerorder = customerorder.rename(columns = {'order_id': 'total_order', 'order_item_id': 'avg_item_ordered'})
customerorder

,customer_unique_id,total_order,avg_item_ordered
0,0000366f3b9a7992bf8c76cfdf3221e2,1,1.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,1.0
2,0000f46a3911fa3c0805444483337064,1,1.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,1.0
4,0004aac84e0df4da2b147fca70cf8255,1,1.0
...,...,...,...
94715,fffcf5a5ff07b0908bd4e2dbc735a684,2,1.5
94716,fffea47cd6d3cc0a88bd621562a9d061,1,1.0
94717,ffff371b4d645b6ecea244b27531430a,1,1.0
94718,ffff5962728ec6157033ef9805bacc48,1,1.0


In [9]:
#most ordered category
cust_order_cat = data[['customer_unique_id','product_category_name']]
cust_order_cat['product_category_name'].fillna("Others",inplace=True)
cust_order_cat['most_cat_ordered'] = cust_order_cat.groupby('customer_unique_id')['product_category_name'].transform(lambda x: x.mode()[0])
cust_order_cat=cust_order_cat.drop(columns=['product_category_name']).drop_duplicates(subset="customer_unique_id").reset_index(drop=True)
cust_order_cat

,customer_unique_id,most_cat_ordered
0,861eff4711a542e4b93843c6dd7febb0,moveis_escritorio
1,9eae34bbd3a474ec5d07949ca7de67c0,moveis_escritorio
2,3c799d181c34d51f6d44bbbc563024db,moveis_escritorio
3,23397e992b09769faf5e66f9e171a241,moveis_escritorio
4,567ab47ca4deb92d46dbf54dce07d0a7,moveis_escritorio
...,...,...
94715,9ba58a98da1d0abf0bd1e422e06c9051,automotivo
94716,5c22dbb9d8b5385953f2f1e6d9d115a7,pet_shop
94717,6532316a8c1a7c7a569482db073530bb,casa_construcao
94718,7f5f43a9394964ac51bc5d1e0204ead4,ferramentas_jardim


In [10]:
#average of how much a customer has paid
avg_spending=data.groupby('customer_unique_id')['customer_id','payment_value'].mean()
avg_spending = avg_spending.rename(columns = {'payment_value' : 'avg_payment'}).reset_index()
avg_spending

,customer_unique_id,avg_payment
0,0000366f3b9a7992bf8c76cfdf3221e2,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,27.19
2,0000f46a3911fa3c0805444483337064,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,43.62
4,0004aac84e0df4da2b147fca70cf8255,196.89
...,...,...
94715,fffcf5a5ff07b0908bd4e2dbc735a684,2067.42
94716,fffea47cd6d3cc0a88bd621562a9d061,84.58
94717,ffff371b4d645b6ecea244b27531430a,112.46
94718,ffff5962728ec6157033ef9805bacc48,133.69


In [11]:
#preferred seller, city, and state
sellercust=data[['customer_unique_id','seller_id','customer_city','customer_state','seller_city','seller_state']]
sellercust['most_seller'] = sellercust.groupby('customer_unique_id')['seller_id'].transform(lambda x: x.mode()[0])
sellercust['most_city'] = sellercust.groupby('customer_unique_id')['seller_city'].transform(lambda x: x.mode()[0])
sellercust['most_state'] = sellercust.groupby('customer_unique_id')['seller_state'].transform(lambda x: x.mode()[0])

#same city/state: yes or no
def same(val1, val2):
    if (val1 == val2):
        return ("Yes")
    else:
        return ("No")
sellercust['same_city']=sellercust.apply(lambda x: same(x.customer_city, x.most_city),axis=1)
sellercust['same_state']=sellercust.apply(lambda x: same(x.customer_state, x.most_state),axis=1)
sellercust=sellercust.drop(columns=['seller_id','customer_city','customer_state','seller_city','seller_state']).drop_duplicates(subset="customer_unique_id").reset_index(drop=True)
sellercust

,customer_unique_id,most_seller,most_city,most_state,same_city,same_state
0,861eff4711a542e4b93843c6dd7febb0,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,Yes
1,9eae34bbd3a474ec5d07949ca7de67c0,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No
2,3c799d181c34d51f6d44bbbc563024db,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No
3,23397e992b09769faf5e66f9e171a241,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No
4,567ab47ca4deb92d46dbf54dce07d0a7,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No
...,...,...,...,...,...,...
94715,9ba58a98da1d0abf0bd1e422e06c9051,4ce6e5f6c52515177e18c1c9361d8677,sao bernardo do campo,SP,No,Yes
94716,5c22dbb9d8b5385953f2f1e6d9d115a7,9140ef75cc8211c4035935e80e567204,maringa,PR,No,No
94717,6532316a8c1a7c7a569482db073530bb,da4d149c0ddbac90557103ac0a0ec356,sbc,SP,No,Yes
94718,7f5f43a9394964ac51bc5d1e0204ead4,377bc18334c78dd2f1535ef22a3ad225,sao jose dos pinhais,PR,No,No


In [12]:
#preferred payment type
custpayment = data.groupby('customer_unique_id')['customer_unique_id','payment_type'].transform(lambda x: x.mode()[0])
custpayment=custpayment.drop_duplicates(subset="customer_unique_id").reset_index(drop=True)
custpayment

,customer_unique_id,payment_type
0,861eff4711a542e4b93843c6dd7febb0,credit_card
1,9eae34bbd3a474ec5d07949ca7de67c0,credit_card
2,3c799d181c34d51f6d44bbbc563024db,credit_card
3,23397e992b09769faf5e66f9e171a241,credit_card
4,567ab47ca4deb92d46dbf54dce07d0a7,credit_card
...,...,...
94715,9ba58a98da1d0abf0bd1e422e06c9051,credit_card
94716,5c22dbb9d8b5385953f2f1e6d9d115a7,credit_card
94717,6532316a8c1a7c7a569482db073530bb,credit_card
94718,7f5f43a9394964ac51bc5d1e0204ead4,credit_card


In [13]:
#avg score given
custrev=data[['customer_unique_id','review_score','review_comment_message','review_creation_date','review_answer_timestamp']]
custrev[['review_creation_date','review_answer_timestamp']]=custrev[['review_creation_date','review_answer_timestamp']].apply(pd.to_datetime)
custrev['review_answer_days']=(custrev['review_answer_timestamp']-custrev['review_creation_date']).dt.days
reviewcust=custrev.groupby('customer_unique_id')['review_score','review_answer_days'].mean().reset_index()
reviewcust=reviewcust.rename(columns={'review_score':'avg_review_score','review_answer_days':'avg_days_rev_answered'})
reviewcust

,customer_unique_id,avg_review_score,avg_days_rev_answered
0,0000366f3b9a7992bf8c76cfdf3221e2,5.0,4.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4.0,0.0
2,0000f46a3911fa3c0805444483337064,3.0,1.0
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,1.0
4,0004aac84e0df4da2b147fca70cf8255,5.0,4.0
...,...,...,...
94715,fffcf5a5ff07b0908bd4e2dbc735a684,5.0,4.0
94716,fffea47cd6d3cc0a88bd621562a9d061,4.0,5.0
94717,ffff371b4d645b6ecea244b27531430a,5.0,2.0
94718,ffff5962728ec6157033ef9805bacc48,5.0,2.0


In [14]:
revleft=data.groupby('customer_unique_id')['review_comment_message'].count().reset_index().rename(columns={'review_comment_message':'review_msg_count'})
revleft

,customer_unique_id,review_msg_count
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,0
2,0000f46a3911fa3c0805444483337064,0
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,0
...,...,...
94715,fffcf5a5ff07b0908bd4e2dbc735a684,0
94716,fffea47cd6d3cc0a88bd621562a9d061,0
94717,ffff371b4d645b6ecea244b27531430a,0
94718,ffff5962728ec6157033ef9805bacc48,1


In [15]:
final = customerdata.merge(activitydata, on ='customer_unique_id',how='left')
final = final.merge(time_to_buy, on ='customer_unique_id',how='left')
final = final.merge(customerorder, on ='customer_unique_id',how='left')
final = final.merge(cust_order_cat, on ='customer_unique_id',how='left')
final = final.merge(avg_spending, on ='customer_unique_id',how='left')
final = final.merge(sellercust, on ='customer_unique_id',how='left')
final = final.merge(custpayment, on ='customer_unique_id',how='left')
final = final.merge(reviewcust, on ='customer_unique_id',how='left')
final = final.merge(revleft, on ='customer_unique_id',how='left')
final

,customer_unique_id,customer_city,customer_state,last_purchase,last_active,activity_status,common_date,common_day,common_month,common_year,...,avg_payment,most_seller,most_city,most_state,same_city,same_state,payment_type,avg_review_score,avg_days_rev_answered,review_msg_count
0,861eff4711a542e4b93843c6dd7febb0,franca,SP,2017-05-16 15:05:35,474,inactive,16,Tue,May,2017,...,146.87,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,Yes,credit_card,4.0,4.0,0
1,9eae34bbd3a474ec5d07949ca7de67c0,santarem,PA,2017-11-09 00:50:13,298,inactive,9,Thu,November,2017,...,275.79,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No,credit_card,1.0,2.0,2
2,3c799d181c34d51f6d44bbbc563024db,nova santa rita,RS,2017-05-07 20:11:26,483,inactive,7,Sun,May,2017,...,140.61,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No,credit_card,3.0,1.0,1
3,23397e992b09769faf5e66f9e171a241,mage,RJ,2018-02-03 19:45:40,211,inactive,3,Sat,February,2018,...,137.58,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No,credit_card,4.0,1.0,1
4,567ab47ca4deb92d46dbf54dce07d0a7,angelina,SC,2017-03-23 15:10:17,528,inactive,23,Thu,March,2017,...,142.05,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,SP,No,No,credit_card,4.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94715,9ba58a98da1d0abf0bd1e422e06c9051,sao manuel,SP,2018-07-16 22:08:56,48,inactive,16,Mon,July,2018,...,252.27,4ce6e5f6c52515177e18c1c9361d8677,sao bernardo do campo,SP,No,Yes,credit_card,3.0,0.0,0
94716,5c22dbb9d8b5385953f2f1e6d9d115a7,jordania,MG,2018-08-01 18:58:49,32,inactive,1,Wed,August,2018,...,72.18,9140ef75cc8211c4035935e80e567204,maringa,PR,No,No,credit_card,5.0,1.0,0
94717,6532316a8c1a7c7a569482db073530bb,guarulhos,SP,2018-08-20 12:37:54,13,active,20,Mon,August,2018,...,775.98,da4d149c0ddbac90557103ac0a0ec356,sbc,SP,No,Yes,credit_card,3.0,1.0,1
94718,7f5f43a9394964ac51bc5d1e0204ead4,uruacu,GO,2018-08-18 20:59:58,15,active,18,Sat,August,2018,...,224.44,377bc18334c78dd2f1535ef22a3ad225,sao jose dos pinhais,PR,No,No,credit_card,5.0,0.0,0
